In [1]:
import keras

Using TensorFlow backend.


In [3]:
from keras.preprocessing.image import ImageDataGenerator as idg
train_data = idg(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
valid_data = idg(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_data = idg(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

In [5]:
training_data = train_data.flow_from_directory('E:\\data\\train', target_size=(32, 32), batch_size=32, class_mode='categorical')
validation_data = valid_data.flow_from_directory('E:\\data\\valid', target_size=(32, 32), batch_size=32, class_mode='categorical')
test_data = test_data.flow_from_directory('E:\\data\\test', target_size=(32, 32), batch_size=32, class_mode='categorical')

Found 2000 images belonging to 3 classes.
Found 150 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


In [34]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu', 
                        input_shape=(32, 32, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=128, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(3, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 32, 32, 32)        416       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 16, 16, 64)        8256      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 8, 8, 128)         32896     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 4, 4, 128)         0         
__________

In [35]:
# compile the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [40]:
from keras.callbacks import ModelCheckpoint   

# train the model
checkpointer = ModelCheckpoint(filepath='derm-best-model', verbose=1, save_best_only=True)
hist = model.fit_generator(training_data, steps_per_epoch=30, epochs=20, verbose=1, callbacks=[checkpointer], 
                           validation_data = (validation_data), validation_steps = 30, shuffle=True)

Epoch 1/20
30/30 [==============================] - 613s 20s/step - loss: 0.7761 - acc: 0.6979 - val_loss: 1.1335 - val_acc: 0.5200

Epoch 00001: val_loss improved from inf to 1.13349, saving model to derm-best-model
Epoch 2/20
30/30 [==============================] - 477s 16s/step - loss: 0.7849 - acc: 0.6792 - val_loss: 1.2062 - val_acc: 0.5200

Epoch 00002: val_loss did not improve from 1.13349
Epoch 3/20
30/30 [==============================] - 472s 16s/step - loss: 0.7860 - acc: 0.6719 - val_loss: 0.8712 - val_acc: 0.5378

Epoch 00003: val_loss improved from 1.13349 to 0.87120, saving model to derm-best-model
Epoch 4/20
30/30 [==============================] - 418s 14s/step - loss: 0.7597 - acc: 0.6906 - val_loss: 0.9249 - val_acc: 0.5356

Epoch 00004: val_loss did not improve from 0.87120
Epoch 5/20
30/30 [==============================] - 439s 15s/step - loss: 0.7329 - acc: 0.7198 - val_loss: 0.9412 - val_acc: 0.5244

Epoch 00005: val_loss did not improve from 0.87120
Epoch 6/20

In [41]:
# load the weights that yielded the best validation accuracy
model.load_weights('derm-best-model')

In [42]:
# evaluate and print test accuracy
score = model.evaluate_generator(test_data, verbose=0)
print('\n', 'Test accuracy:', score[1])


 Test accuracy: 0.701666666667


In [43]:
predictions = model.predict_generator(test_data, verbose=0)

In [44]:
print(predictions)

[[ 0.20775467  0.73647171  0.05577361]
 [ 0.27240661  0.45568585  0.27190751]
 [ 0.32488066  0.60155147  0.07356779]
 ..., 
 [ 0.33580443  0.39660552  0.26758999]
 [ 0.23638368  0.64653587  0.11708048]
 [ 0.31824487  0.64059627  0.04115896]]
